# 循环神经网络
:label:`sec_rnn`

>RNN参数更新可视化
- [RNN参数更新可视化](https://www.bilibili.com/video/BV1z5411f7Bm)


在 :numref:`sec_language_model`中，
我们介绍了$n$元语法模型，
其中单词$x_t$在时间步$t$的**条件概率仅取决于前面$n-1$个单词**。
对于时间步$t-(n-1)$之前的单词，
如果我们想将其可能产生的影响合并到$x_t$上，
需要增加$n$，然而模型参数的数量也会随之呈指数增长，
因为词表$\mathcal{V}$需要存储$|\mathcal{V}|^n$个数字，
因此与其将$P(x_t \mid x_{t-1}, \ldots, x_{t-n+1})$模型化，
不如使用隐变量模型：

$$P(x_t \mid x_{t-1}, \ldots, x_1) \approx P(x_t \mid h_{t-1}),$$

其中$h_{t-1}$是*隐状态*（hidden state），
也称为*隐藏变量*（hidden variable），
它存储了到时间步$t-1$的序列信息。
通常，我们可以基于当前输入$x_{t}$和先前隐状态$h_{t-1}$
来计算时间步$t$处的任何时间的隐状态：

$$h_t = f(x_{t}, h_{t-1}).$$
:eqlabel:`eq_ht_xt`

对于 :eqref:`eq_ht_xt`中的函数$f$，隐变量模型不是近似值。
毕竟$h_t$是可以仅仅存储到目前为止观察到的所有数据，
然而这样的操作可能会使计算和存储的代价都变得昂贵。

回想一下，我们在 :numref:`chap_perceptrons`中
讨论过的具有隐藏单元的隐藏层。
值得注意的是，**隐藏层(hidden layer)和隐状态(hidden state)**指的是两个截然不同的概念。
如上所述，隐藏层(hidden layer)是在从输入到输出的路径上（以观测角度来理解）的隐藏的层，
而隐状态(hidden state)则是在给定步骤所做的任何事情（以技术角度来定义）的*输入*，
并且这些状态只能通过先前时间步的数据来计算。

*循环神经网络*（recurrent neural networks，RNNs）
是具有隐状态的神经网络。
在介绍循环神经网络模型之前，
我们首先回顾 :numref:`sec_mlp`中介绍的多层感知机模型。

## 无隐状态的神经网络

让我们来看一看只有单隐藏层的**多层感知机**。
设隐藏层的激活函数为$\phi$，
给定一个小批量样本$\mathbf{X} \in \mathbb{R}^{n \times d}$，
其中批量大小为$n$，输入维度为$d$，
则隐藏层的输出$\mathbf{H} \in \mathbb{R}^{n \times h}$通过下式计算：

$$\mathbf{H} = \phi(\mathbf{X} \mathbf{W}_{xh} + \mathbf{b}_h).$$
:eqlabel:`rnn_h_without_state`

在 :eqref:`rnn_h_without_state`中，
我们拥有的隐藏层权重参数为$\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}$，
偏置参数为$\mathbf{b}_h \in \mathbb{R}^{1 \times h}$，
以及隐藏单元的数目为$h$。
因此求和时可以应用广播机制（见 :numref:`subsec_broadcasting`）。
接下来，将隐藏变量$\mathbf{H}$用作输出层的输入。
输出层由下式给出：

$$\mathbf{O} = \mathbf{H} \mathbf{W}_{hq} + \mathbf{b}_q,$$

其中，$\mathbf{O} \in \mathbb{R}^{n \times q}$是输出变量，
$\mathbf{W}_{hq} \in \mathbb{R}^{h \times q}$是权重参数，
$\mathbf{b}_q \in \mathbb{R}^{1 \times q}$是输出层的偏置参数。
如果是分类问题，我们可以用$\text{softmax}(\mathbf{O})$
来计算输出类别的概率分布。

这完全类似于之前在 :numref:`sec_sequence`中解决的回归问题，
因此我们省略了细节。
无须多言，只要可以随机选择“特征-标签”对，
并且通过自动微分和随机梯度下降能够学习网络参数就可以了。



## 有隐状态的循环神经网络
:label:`subsec_rnn_w_hidden_states`

有了隐状态后，情况就完全不同了。

假设我们在时间步$t$有小批量输入$\mathbf{X}_t \in \mathbb{R}^{n \times d}$。
换言之，对于$n$个序列样本的小批量，
$\mathbf{X}_t$的每一行对应于来自该序列的时间步$t$处的一个样本。

接下来，用$\mathbf{H}_t  \in \mathbb{R}^{n \times h}$
表示时间步$t$的隐藏变量。
与多层感知机不同的是，
我们在这里保存了前一个时间步的隐藏变量$\mathbf{H}_{t-1}$，
并引入了一个新的权重参数$\mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$，
来描述如何在当前时间步中使用前一个时间步的隐藏变量。

具体地说，**当前时间步隐藏变量由当前时间步的输入与前一个时间步的隐藏变量一起计算得出**：

$$\mathbf{H}_t = \phi(\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}  + \mathbf{b}_h).$$
:eqlabel:`rnn_h_with_state`

与 :eqref:`rnn_h_without_state`相比，
 :eqref:`rnn_h_with_state`多添加了一项
$\mathbf{H}_{t-1} \mathbf{W}_{hh}$，
从而实例化了$h_t = f(x_{t}, h_{t-1})$, 即显示定义了$x_{t}$,$h_{t-1}$得到$h_{t}$的关系。

从相邻时间步的隐藏变量$\mathbf{H}_t$和
$\mathbf{H}_{t-1}$之间的关系可知，
这些**变量捕获并保留了序列直到其当前时间步的历史信息**，
> 但这些变量$\mathbf{H}_t$旨在捕获并传递序列截至当前时间步的历史信息。然而，由于隐状态通过权重系数$\mathbf{W}_{hh}$递归更新，早期时间步的信息在**多次矩阵乘积作用下会指数级衰减（或爆炸）**，导致实际有效记忆长度受限，远古信息往往被严重削弱甚至丢失。”

就如当前时间步下神经网络的状态或记忆，
因此这样的隐藏变量被称为*隐状态*（hidden state）。
由于在当前时间步中，
隐状态使用的定义与前一个时间步中使用的定义相同，
因此 :eqref:`rnn_h_with_state`的计算是*循环的*（recurrent）。
于是基于循环计算的隐状态神经网络被命名为
*循环神经网络*（recurrent neural network）。
在循环神经网络中执行 :eqref:`rnn_h_with_state`**hidden state renewed**计算的层
称为*循环层*（recurrent layer）。

有许多不同的方法可以构建循环神经网络，
由 :eqref:`rnn_h_with_state`定义的隐状态的循环神经网络是非常常见的一种。
对于时间步$t$，输出层的输出类似于多层感知机中的计算：

$$\mathbf{O}_t = \mathbf{H}_t \mathbf{W}_{hq} + \mathbf{b}_q.$$

循环神经网络的参数包括隐藏层的权重
$\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}, \mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$和偏置$\mathbf{b}_h \in \mathbb{R}^{1 \times h}$，
以及输出层的权重$\mathbf{W}_{hq} \in \mathbb{R}^{h \times q}$
和偏置$\mathbf{b}_q \in \mathbb{R}^{1 \times q}$。
值得一提的是，**即使在不同的时间步，循环神经网络也总是使用这些模型参数**。
因此，循环神经网络的参数开销不会随着时间步的增加而增加。

 :numref:`fig_rnn`展示了循环神经网络在三个相邻时间步的计算逻辑。
在任意时间步$t$，隐状态(hidden state)的计算可以被视为：

1. 拼接当前时间步$t$的输入$\mathbf{X}_t$和前一时间步$t-1$的隐状态$\mathbf{H}_{t-1}$；
1. 将拼接的结果送入带有激活函数$\phi$的全连接层。
   全连接层的输出是当前时间步$t$的隐状态$\mathbf{H}_t$。
   
在本例中，模型参数是$\mathbf{W}_{xh}$和$\mathbf{W}_{hh}$的拼接，
以及$\mathbf{b}_h$的偏置，所有这些参数都来自 :eqref:`rnn_h_with_state`。
当前时间步$t$的隐状态$\mathbf{H}_t$
将参与计算下一时间步$t+1$的隐状态$\mathbf{H}_{t+1}$。
而且$\mathbf{H}_t$还将送入全连接输出层，
用于计算当前时间步$t$的输出$\mathbf{O}_t$。

![具有隐状态的循环神经网络](../img/rnn.svg)
:label:`fig_rnn`

我们刚才提到，隐状态中
$\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}$的计算，
相当于$\mathbf{X}_t$和$\mathbf{H}_{t-1}$的拼接
与$\mathbf{W}_{xh}$和$\mathbf{W}_{hh}$的拼接的矩阵乘法。
虽然这个性质可以通过数学证明，
但在下面我们使用一个简单的代码来说明一下。
首先，我们定义矩阵`X`、`W_xh`、`H`和`W_hh`，
它们的形状分别为$(3，1)$、$(1，4)$、$(3，4)$和$(4，4)$。
分别将`X`乘以`W_xh`，将`H`乘以`W_hh`，
然后将这两个乘法相加，我们得到一个形状为$(3，4)$的矩阵。


#### 在 RNN（循环神经网络）的语境中，“**隐变量**（latent variable）”和“**隐藏状态**（hidden state）”经常被混用，但它们在**概念层次**和**建模目的**上有重要区别。

下面我将从**定义、角色、可观察性、建模范式**等多个维度详细解释，并给出清晰例子。

🔍 一、核心概念区分

| 术语 | 中文 | 英文 | 概念层级 | 是否可学习/推断 | 是否显式存在于模型中 |
|------|------|------|----------|------------------|---------------------|
| **隐藏状态** | 隐藏状态 / 隐状态 | Hidden State | **模型内部机制** | 是（通过反向传播直接优化） | ✅ 是（RNN 的核心组件） |
| **隐变量** | 潜在变量 / 隐变量 | Latent Variable | **生成模型假设** | 通常需**推断**（inference） | ❌ 不一定显式存在（可能需变分推断） |

> 💡 **一句话总结**：  
> - **Hidden state 是 RNN 的“工作记忆”** —— 它是确定性的、由输入和前一状态计算得出。  
> - **Latent variable 是对数据生成机制的“未知原因”的建模** —— 它是概率性的、代表我们无法观测的潜在因素。


🧠 二、详细解释

### 1. **Hidden State（隐藏状态）—— RNN 的标准组件**

- **定义**：RNN 在时间步 $t$ 的内部状态 $h_t$，由下式计算：
  $$
  h_t = \text{RNNCell}(x_t, h_{t-1}; \theta)
  $$
  其中 $\theta$ 是可学习参数。

- **性质**：
  - **确定性**：给定 $x_{1:t}$ 和初始 $h_0$，$h_t$ 唯一确定
  - **可微分**：可通过 BPTT（Backpropagation Through Time）端到端训练
  - **用途**：用于预测 $y_t$（如 $y_t = \text{softmax}(W h_t)$）

- **例子**（标准 RNN 语言模型）：
  ```python
  # 输入: x = [w1, w2, w3]
  h1 = rnn_cell(w1, h0)   # h1 是 hidden state
  h2 = rnn_cell(w2, h1)   # h2 依赖 h1
  h3 = rnn_cell(w3, h2)
  prob_next = softmax(W @ h3)  # 预测下一个词
  ```
  → 这里 **没有 latent variable**，只有 deterministic hidden state。


### 2. **Latent Variable（隐变量）—— 生成模型中的随机因素**

- **定义**：假设观测序列 $x_{1:T}$ 是由一个**不可观测的随机过程** $z_{1:T}$ 生成的：
  $$
  z_t \sim p(z_t \mid z_{t-1}), \quad x_t \sim p(x_t \mid z_t)
  $$
  其中 $z_t$ 就是 **latent variable**。

- **性质**：
  - **随机性**：$z_t$ 是随机变量，有概率分布
  - **不可观测**：只能通过 $x_{1:T}$ 间接推断
  - **建模目标**：学习生成过程 $p(x_{1:T}) = \int p(x_{1:T}, z_{1:T}) dz$

- **与 RNN 的结合**：  
  可用 RNN 作为 **近似推断网络**（inference network）或 **生成网络**（generative network）。

---

🧪 三、实战对比：两个具体例子

### ✅ 例 1：标准 RNN（只有 hidden state，无 latent variable）

**任务**：字符级语言模型（预测下一个字符）

```python
import torch
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = nn.RNN(vocab_size, hidden_dim, batch_first=True)
        self.proj = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):  # x: (B, T, V)
        h, _ = self.rnn(x)          # h: (B, T, H) ← hidden states
        logits = self.proj(h)       # (B, T, V)
        return logits
```

- **Hidden state `h`**：
  - 是确定性向量
  - 直接用于预测
  - **不是 latent variable**（因为没有随机性，也不代表“潜在原因”）

> 📌 这是**判别式模型**（discriminative）：直接建模 $p(y_t \mid x_{1:t})$



### ✅ 例 2：VRNN（Variational RNN）—— 同时包含 hidden state 和 latent variable

**任务**：生成手写轨迹（如 IAM-OnDB），数据有多种书写风格

#### 模型结构（简化）：
- **Latent variable** $z_t$：表示时间步 $t$ 的“书写风格”（如笔画粗细、倾斜度）
- **Hidden state** $h_t$：RNN 的内部状态，用于传递信息

#### 生成过程：
1. $h_t^{\text{prior}} = \text{RNN}_{\text{prior}}(h_{t-1}, z_{t-1})$
2. $z_t \sim \mathcal{N}(\mu_t, \sigma_t)$，其中 $\mu_t, \sigma_t = \text{MLP}(h_t^{\text{prior}})$
3. $x_t \sim \mathcal{N}(W z_t + U h_t^{\text{gen}}, \text{const})$

#### 推断过程（用另一个 RNN）：
- $h_t^{\text{enc}} = \text{RNN}_{\text{enc}}(h_{t-1}^{\text{enc}}, x_{t-1})$
- 用 $x_t$ 和 $h_t^{\text{enc}}$ 推断 $q(z_t \mid x_{\leq t})$

#### 关键区别：
| 组件 | 角色 | 是否随机 |
|------|------|--------|
| $z_t$ | **Latent variable**：表示无法观测的书写风格 | ✅ 随机采样 |
| $h_t$ | **Hidden state**：RNN 的确定性记忆 | ❌ 确定性计算 |

> 📌 这是**生成式模型**（generative）：建模 $p(x_{1:T})$，并引入 latent variable 解释数据多样性。

---

🆚 四、对比总结表

| 特性 | Hidden State（隐藏状态） | Latent Variable（隐变量） |
|------|------------------------|-------------------------|
| **存在位置** | 所有 RNN 中都有 | 仅在**生成模型**（如 VAE、SSM）中出现 |
| **确定性** | 确定性（函数计算） | 随机性（概率分布采样） |
| **可观察性** | 模型内部可见 | 不可观测，需推断 |
| **训练方式** | 直接反向传播 | 需变分推断 / EM / 重参数化技巧 |
| **目的** | 传递历史信息 | 解释数据生成的潜在原因 |
| **例子** | 标准 RNN、LSTM、GRU | VRNN、Kalman Filter、Deep State Space Models |

💡 五、为什么容易混淆？

1. **中文翻译问题**：
   - “hidden” 和 “latent” 在中文都常译作“隐”或“潜”
   - 但英文中 **hidden ≠ latent**

2. **LSTM/GRU 的“cell state”有时被误称为 latent**：
   - 实际上它仍是 deterministic hidden state

3. **在简单 RNN 中，hidden state 起到了“压缩历史”的作用，看似像 latent**：
   - 但本质不同：latent 是**对生成机制的假设**，hidden 是**计算工具**

---

## ✅ 六、一句话记住区别

> - **Hidden state 是 RNN 的“内存”** —— 它记住过去，帮助预测未来。  
> - **Latent variable 是世界的“秘密”** —— 它解释为什么数据长这样，但我们看不见它。


🌟 附加：何时需要 latent variable？

当你遇到以下情况时，考虑引入 latent variable：
- 数据有**多种生成模式**（如不同说话人语音）
- 观测是**噪声投影**（如传感器融合）
- 需要**生成多样样本**（如文本、图像）
- 想建模**不确定性**（如机器人路径规划）

而在大多数**判别任务**（分类、翻译、NER）中，**hidden state 已足够**，无需 latent variable。

In [1]:
import torch
from myd2l import torch as d2l

In [2]:
X, W_xh = torch.normal(0, 1, (3, 1)), torch.normal(0, 1, (1, 4))
H, W_hh = torch.normal(0, 1, (3, 4)), torch.normal(0, 1, (4, 4))
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[-1.2708, -0.7724,  0.2343,  0.3928],
        [ 2.0955,  0.3975, -0.9012, -0.3609],
        [ 1.8551, -2.9493, -3.7219,  2.2094]])

现在，我们沿列（轴1）拼接矩阵`X`和`H`，
沿行（轴0）拼接矩阵`W_xh`和`W_hh`。
这两个拼接分别产生形状$(3, 5)$和形状$(5, 4)$的矩阵。
再将这两个拼接的矩阵相乘，
我们得到与上面相同形状$(3, 4)$的输出矩阵。


In [3]:
torch.matmul(torch.cat((X, H), 1), torch.cat((W_xh, W_hh), 0))

tensor([[-1.2708, -0.7724,  0.2343,  0.3928],
        [ 2.0955,  0.3975, -0.9012, -0.3609],
        [ 1.8551, -2.9493, -3.7219,  2.2094]])

## 基于循环神经网络的字符级语言模型

回想一下 :numref:`sec_language_model`中的语言模型，
我们的目标是根据过去的和当前的词元预测下一个词元，
因此我们将原始序列移位一个词元作为标签。
Bengio等人首先提出使用神经网络进行语言建模
 :cite:`Bengio.Ducharme.Vincent.ea.2003`。
接下来，我们看一下如何使用循环神经网络来构建语言模型。
设小批量大小为1，批量中的文本序列为“machine”。
为了简化后续部分的训练，我们考虑使用
*字符级语言模型*（character-level language model），
将文本词元化为**字符 char**而不是单词。
 :numref:`fig_rnn_train`演示了
如何通过基于字符级语言建模的循环神经网络，
使用当前的和先前的字符预测下一个字符。

![基于循环神经网络的字符级语言模型：输入序列和标签序列分别为“machin”和“achine”](../img/rnn-train.svg)
:label:`fig_rnn_train`

在训练过程中，我们对每个时间步的输出层的输出进行softmax操作，
然后利用交叉熵损失计算模型输出和标签之间的误差。
由于隐藏层中隐状态的循环计算，
 :numref:`fig_rnn_train`中的第$3$个时间步的输出$\mathbf{O}_3$
由文本序列“m”“a”和“c”确定。
由于训练数据中这个文本序列的下一个字符是“h”，
因此第$3$个时间步的损失将取决于下一个字符的概率分布，
而下一个字符是基于特征序列“m”“a”“c”和这个时间步的标签“h”生成的。

在实践中，我们使用的批量大小为$n>1$，
每个词元都由一个$d$维向量表示。
因此，在时间步$t$输入$\mathbf X_t$将是一个$n\times d$矩阵，
这与我们在 :numref:`subsec_rnn_w_hidden_states`中的讨论相同。




> “下一个字符是基于特征序列‘m’‘a’‘c’和这个时间步的标签‘h’生成的”

⚠️ **这句话表述容易引起误解！**  
它不是说“标签生成了字符”，而是想表达：

> **模型基于输入序列 `"m","a","c"` 生成对下一个字符的预测分布，而训练时用真实标签 `"h"` 来计算损失。**

正确理解应为：
- **输入**（特征）：`x₁='m', x₂='a', x₃='c'`
- **模型内部**：通过 RNN 计算出 $h_3$，再得到预测分布 $\hat{y}_3$
- **标签**（监督信号）：$y_3 = \text{‘h’}$（来自数据）
- **损失**：衡量 $\hat{y}_3$ 与 one-hot($y_3$) 的差距

> ❌ **不是**：“标签生成了字符”  
> ✅ **而是**：“标签告诉我们正确答案是什么，用来评估模型预测的好坏”

---

🧩 用图示辅助理解（对应 :numref:`fig_rnn_train`）

```
时间步 t:    1       2       3       4       5       6
输入 X:      'm' → 'a' → 'c' → 'h' → 'i' → 'n'
                    ↓     ↓     ↓     ↓     ↓     ↓
隐藏状态:    h₁    h₂    h₃    h₄    h₅    h₆
                    ↓     ↓     ↓     ↓     ↓     ↓
输出 O:      o₁     o₂     o₃     o₄     o₅     o₆
                    ↓     ↓     ↓     ↓     ↓     ↓
预测 ŷ:      ŷ₁      ŷ₂      ŷ₃      ŷ₄      ŷ₅      ŷ₆
                    ↘     ↘     ↘     ↘     ↘     ↘
标签 Y:             'a'    'c'    'h'    'i'    'n'    'e'
```

- 在 **t=3**：
  - 模型看到 `"m","a","c"` → 输出 $\hat{y}_3$
  - 真实下一个字符是 `"h"` → 标签 $y_3 = \text{‘h’}$
  - 损失 $\mathcal{L}_3 = -\log P(\text{‘h’} \mid \text{‘m’,‘a’,‘c’})$

---

## ✅ 总结：如何正确理解这句话？

> **更清晰的重述**：  
> “因为在训练数据中，字符序列 ‘m’, ‘a’, ‘c’ 的下一个字符是 ‘h’，  
> 所以在第 3 个时间步，模型应预测 ‘h’。  
> 模型基于输入 ‘m’, ‘a’, ‘c’ 生成一个**所有可能字符的概率分布**，  
> 然后用**真实标签 ‘h’** 与该分布计算交叉熵损失，从而指导参数更新。”

---

## 💡 附加思考：为什么用“概率分布”而不是直接预测？

- 语言具有**不确定性**：即使上下文相同，也可能有多个合理后续（如 “I love ___” → “you”, “chocolate”, “AI”）
- Softmax + 交叉熵 允许模型：
  - 表达不确定性
  - 通过梯度平滑更新（而非硬分类）
  - 支持采样生成（如 temperature sampling）

---

你现在不仅理解了这句话的真实含义，还掌握了**RNN 语言模型训练的核心机制**：  
**用历史预测未来，用真实未来监督预测，用概率建模不确定性**。

## 困惑度（Perplexity）
:label:`subsec_perplexity`

最后，让我们讨论如何度量语言模型的质量，
这将在后续部分中用于评估基于循环神经网络的模型。
一个好的语言模型能够用高度准确的词元来预测我们接下来会看到什么。
考虑一下由不同的语言模型给出的对“It is raining ...”（“...下雨了”）的续写：

1. "It is raining outside"（外面下雨了）；
1. "It is raining banana tree"（香蕉树下雨了）；
1. "It is raining piouw;kcj pwepoiut"（piouw;kcj pwepoiut下雨了）。

就质量而言，例$1$显然是最合乎情理、在逻辑上最连贯的。
虽然这个模型可能没有很准确地反映出后续词的语义，
比如，“It is raining in San Francisco”（旧金山下雨了）
和“It is raining in winter”（冬天下雨了）
可能才是更完美的合理扩展，
但该模型已经能够捕捉到跟在后面的是哪类单词。
例$2$则要糟糕得多，因为其产生了一个无意义的续写。
尽管如此，至少该模型已经学会了如何拼写单词，
以及单词之间的某种程度的相关性。
最后，例$3$表明了训练不足的模型是无法正确地拟合数据的。

我们可以通过计算序列的似然概率来度量模型的质量。
然而这是一个难以理解、难以比较的数字。
毕竟，较短的序列比较长的序列更有可能出现，
因此评估模型产生托尔斯泰的巨著《战争与和平》的可能性
不可避免地会比产生圣埃克苏佩里的中篇小说《小王子》可能性要小得多。
而缺少的可能性值相当于平均数。

在这里，信息论可以派上用场了。
我们在引入softmax回归
（ :numref:`subsec_info_theory_basics`）时定义了熵、惊异和交叉熵，
并在[信息论的在线附录](https://d2l.ai/chapter_appendix-mathematics-for-deep-learning/information-theory.html)
中讨论了更多的信息论知识。
如果想要压缩文本，我们可以根据当前词元集预测的下一个词元。
一个更好的语言模型应该能让我们更准确地预测下一个词元。
因此，它应该允许我们在压缩序列时花费更少的比特。
所以我们可以通过一个序列中所有的$n$个词元的交叉熵损失的平均值来衡量：

$$\frac{1}{n} \sum_{t=1}^n -\log P(x_t \mid x_{t-1}, \ldots, x_1),$$
:eqlabel:`eq_avg_ce_for_lm`

其中$P$由语言模型给出，
$x_t$是在时间步$t$从该序列中观察到的实际词元。
这使得不同长度的文档的性能具有了可比性。
由于历史原因，自然语言处理的科学家更喜欢使用一个叫做*困惑度*（perplexity）的量。
简而言之，它是 :eqref:`eq_avg_ce_for_lm`的指数：

$$\exp\left(-\frac{1}{n} \sum_{t=1}^n \log P(x_t \mid x_{t-1}, \ldots, x_1)\right).$$

困惑度的最好的理解是“**下一个词元的实际选择数的调和平均数**”。
我们看看一些案例。

* 在最好的情况下，模型总是完美地估计标签词元的概率为1。
  在这种情况下，模型的困惑度为1。
* 在最坏的情况下，模型总是预测标签词元的概率为0。
  在这种情况下，困惑度是正无穷大。
* 在基线上，该模型的预测是词表的所有可用词元上的均匀分布。
  在这种情况下，**困惑度等于词表中唯一词元的数量**。
  事实上，如果我们在没有任何压缩的情况下存储序列，
  这将是我们能做的最好的编码方式。
  因此，这种方式提供了一个重要的上限，
  而任何实际模型都必须超越这个上限。

在接下来的小节中，我们将基于循环神经网络实现字符级语言模型，
并使用困惑度来评估这样的模型。

## 小结

* 对隐状态使用循环计算的神经网络称为循环神经网络（RNN）。
* 循环神经网络的隐状态可以捕获直到当前时间步序列的历史信息。
* 循环神经网络模型的参数数量不会随着时间步的增加而增加。
* 我们可以使用循环神经网络创建字符级语言模型。
* 我们可以使用困惑度来评价语言模型的质量。

## 练习

1. 如果我们使用循环神经网络来预测文本序列中的下一个字符，那么任意输出所需的维度是多少？
1. 为什么循环神经网络可以基于文本序列中所有先前的词元，在某个时间步表示当前词元的条件概率？
1. 如果基于一个长序列进行反向传播，梯度会发生什么状况？
1. 与本节中描述的语言模型相关的问题有哪些？


[Discussions](https://discuss.d2l.ai/t/2100)


1. 如果我们使用循环神经网络来预测文本序列中的下一个字符，那么任意输出所需的维度是多少？

---

你的回答：
> “维度应该等于语料库中所有 token 的数量 n”

🔍 点评：
- **✅ 正确！**
- 更精确地说：**输出维度 = 词表大小（vocabulary size）**，记作 $ | \mathcal{V} | $
- 因为模型需对**每个可能的下一个 token** 输出一个 logits 或概率

📌 建议优化表述：
> “输出层的维度等于词表中唯一 token 的数量 $ |\mathcal{V}| $，因为模型需要为每个可能的下一个字符（或词元）计算一个未归一化的得分（logit），后续通过 softmax 转换为概率分布。”

> 💡 注意：这里“token”可以是字符（char-level）或单词（word-level），但维度始终是 **词表大小**。
- **第1问延伸**：若使用 subword（如 BPE），维度 = subword 词表大小

1. 为什么循环神经网络可以基于文本序列中所有先前的词元，在某个时间步表示当前词元的条件概率？

---

你的回答：
> “因为 RNN 在 FP 过程中传递了基于历史信息的 hidden state”

🔍 点评：
- **✅ 核心思想正确！**
- “FP” 应写全称 “forward propagation”（前向传播）
- 可进一步说明：**隐状态 $h_t$ 是历史输入的函数**，从而使得 $P(x_{t+1} \mid x_{1:t})$ 可被建模

📌 建议优化表述：
> “RNN 通过递归更新隐藏状态 $h_t = f(x_t, h_{t-1})$，使得 $h_t$ 编码了从时间步 1 到 $t$ 的所有输入信息。因此，在时间步 $t$，输出层可基于 $h_t$ 计算下一个词元的条件概率 $P(x_{t+1} \mid x_1, x_2, ..., x_t)$。”

> ⚠️ 注意：虽然理论上 $h_t$ 包含全部历史，但实际上受梯度消失影响，**长期依赖难以保留**（可提也可不提，看上下文）。
- **第2问延伸**：严格来说，RNN 建模的是 **马尔可夫近似** $P(x_{t+1} \mid h_t) \approx P(x_{t+1} \mid x_{1:t})$

1. 如果基于一个长序列进行反向传播，梯度会发生什么状况？

---
你的回答：
> “梯度消失或者爆炸”

🔍 点评：
- **✅ 完全正确！这是 RNN 最著名的缺陷**
- 可补充原因：由于链式法则中连续乘以 $\frac{\partial h_t}{\partial h_{t-1}} \approx \mathbf{W}_{hh}$，导致梯度呈指数级衰减或增长

📌 建议优化表述：
> “在长序列上进行反向传播时，梯度会通过多个时间步连续相乘（涉及权重矩阵 $\mathbf{W}_{hh}$ 的幂次）。若其最大特征值小于 1，梯度会指数级衰减（梯度消失）；若大于 1，则会指数级增长（梯度爆炸）。这使得 RNN 难以学习长期依赖。”
- **第3问延伸**：LSTM/GRU 通过门控缓解此问题；Transformer 用自注意力彻底避免